In [1]:
from dotenv import load_dotenv
from pathlib import Path
import os
import sys
import openai
from typing import Optional, Union


env_path = Path('../..') / '.env'
load_dotenv(dotenv_path=env_path)

print(os.getcwd())
sys.path.append('/Users/davidwei/Documents/SellScale/sellscale-api/notebooks/research')

OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
openai.api_key = OPENAI_API_KEY

/Users/davidwei/Documents/SellScale/sellscale-api/notebooks/rules_engine


In [47]:
import json


def detect_hallucinations(message_prompt: str, message: str):
    system_instructions = "You are an assistant that will help me detect hallucinations. A hallucination is defined as messaging that references entities that were not present in the original prompt."
    prompt = """Help me determine if there are any hallucinations in the following generated message. A hallucination is defined as messaging that references entities that were not present in the original prompt.

==== START PROMPT ====
{message_prompt}
==== END PROMPT ====

==== START MESSAGE ====
{message}
==== END MESSAGE ====

Please return a JSON object such as the following:
{{
    "hallucinations": ["Apple", "U.S. Air Force"]
}}
Just return the JSON string object, no prose. Do not include ```json. We will use json.loads().

If there are no hallucinations, just return an empty list

Output:
""".format(
    message_prompt=message_prompt,
    message=message
)
    
    
    response = openai.ChatCompletion.create(
        model='gpt-4-turbo-preview',
        messages=[
            {"role": "system", "content": system_instructions},
            {"role": "user", "content": prompt},
        ],
        max_tokens=100,
        stop=None
    )
    if response is None or response["choices"] is None or len(response["choices"]) == 0:
        return ""

    choices = response["choices"]
    top_choice = choices[0]
    preview = top_choice["message"]["content"].strip()
    print(preview)
    
    preview = json.loads(preview)
    hallucinations = preview.get("hallucinations", [])

    return hallucinations

In [48]:
detect_hallucinations("talk about Apple", "Facebook is a tech company")

{
    "hallucinations": ["Facebook"]
}


['Facebook']